# TODO
> PICK UP HERE: **implement model end-to-end ASAP using Dataset, saving model state every N epochs, etc.**
- See todo list in OneNote
- Once working, make the saving model state logic generic so I can use with any model

In [165]:
from datatype_recovery.models.dataset import TypeSequenceDataset

data_params = {
    'experiment_runs': [
        '/home/cls0027/test_builds/astera.exp/rundata/run1',
        '/home/cls0027/test_builds/coreutils.exp/rundata/run1',
    ],
    'copy_data': False,
}
dataset = TypeSequenceDataset('model0_dataset', data_params, max_hops=3)

input_params will be IGNORED in favor of saved .json file (model0_dataset/raw/input_params.json)


In [166]:
# TODO: split test/train on BINARY boundary to prevent functions/vars from crossing
# -> I have varid tuples, so I can get (rungid, binid) from that and use it to split
#   - extract (idx, rungid, binid) for each data point and put into a dataframe
#   - groupby/count dataframe by (rungid/binid) and come up with split that is close to target %
#   - collect indices for each data point within each subset (test/train/valid)
#   - quick/simple validate that all (rungid/binid) combos are unique to a subset
#
# [(i, x.varid[0], x.varid[1]) for i, x in enumerate(dataset[10000:10100])][:5]

# trainset_indices = list(range(12))
# testset_indices = list(range(100,112))
# train_set = Subset(dataset, trainset_indices)
# test_set = Subset(dataset, testset_indices)


from torch_geometric.loader import DataLoader
from torch.utils.data import Subset

# tiny subset to start
TRAIN_SPLIT = 0.7
BATCH_SIZE = 64
SHUFFLE = False
PIN_MEMORY=True

dataset = dataset[:1000]

# divide into train/test sets
train_size = int(len(dataset)*TRAIN_SPLIT)
train_size -= train_size%BATCH_SIZE     # align to batch size

test_size = len(dataset) - train_size   # take the rest of the dataset...
test_size -= test_size % BATCH_SIZE     # align to batch size

train_set = Subset(dataset, range(0, 0+train_size))
test_set = Subset(dataset, range(len(train_set), len(train_set)+test_size))

train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=SHUFFLE, pin_memory=PIN_MEMORY)
test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=SHUFFLE, pin_memory=PIN_MEMORY)

# EXAMPLE: the shuffled slice changes each time, but dataset[0] stays consistent
# print(dataset[:10].shuffle()[0].varid)
# print(dataset[0].varid)
total_usable = len(train_set)+len(test_set)
non_batch_aligned = len(dataset)-total_usable
print(f'Train set: {len(train_set):,} samples ({len(train_set)/len(dataset)*100:.2f}%)')
print(f'Test set: {len(test_set):,} samples ({len(test_set)/len(dataset)*100:.2f}%)')
print(f'Batch size: {BATCH_SIZE}')
print(f'Total usable dataset size (batch-aligned): {total_usable:,}')
print(f'Loss due to batch alignment: {non_batch_aligned:,} ({non_batch_aligned/len(dataset)*100:.2f}%)')

Train set: 640 samples (64.00%)
Test set: 320 samples (32.00%)
Batch size: 64
Total usable dataset size (batch-aligned): 960
Loss due to batch alignment: 40 (4.00%)


In [167]:
# for step, data in enumerate(train_loader):
#     if step > 5:
#         break
#     print(f'Step {step+1}')
#     print(f'Num graphs in batch: {data.num_graphs}')
#     print(data)
#     print(data.batch)
#     print()

# Handle Batches
This code 

In [168]:
# or more elegantly...

# b = list(test_loader)[0]
# batch_graphs = [(b.batch==i).nonzero(as_tuple=True) for i in range(BATCH_SIZE)]

# node0_indices = [x[0][0].item() for x in batch_graphs]
# node0_indices
# batch_graphs
# batch shape: [76, 47]
# - 76 = sum of nodes in all graphs within the batch = 11+21+17+27
# - 47 = # node features
# batch_graphs
# list of tensors, one tensor per graph within the batch
# --> list of node_indexes, one per graph within the batch
# each tensor holds a list of node indices for the nodes of that graph
# (the indices index into b.x)
# indices of nodes

import torch
from torch_geometric.data.batch import DataBatch
from typing import List

def split_node_index_by_graph(batch:torch.tensor, batch_size:int) -> List[torch.tensor]:
    '''
    Returns the node_index for each individual graph within the batch as a list of tensors.

    Each list entry (node_index) is a tensor of node indices for an individual graph within the
    batch, where each of the node indices will index into the containing DataBatch.x.
    '''
    return [(batch==i).nonzero(as_tuple=True) for i in range(batch_size)]


# c = list(loader2)[:4]
# for x in list(test_loader)[:4]:
    # print(x)

# len(b.batch)
# list(loader2)[3].edge_index
# b.edge_index
# b.batch[(b.batch==0).nonzero(as_tuple=True)]



# ------------------------
# TODO: ok, here is the way to get node 0 indices
# - edge_index should be able to take care of itself - we just pass it through, I
#   don't think we need to do anything to it (it's correct, but won't have edges
#   going between independent graphs of course)
# ------------------------
# TODO: PICK UP HERE and IMPLEMENT THE MODEL!!!
# ------------------------
# - use node0_indices to pull target node embeddings, passing into final layer (linear for now?)
# - return prediction...finish building training function
# - INTEGRATE WANDB!!
# ------------------------

def get_node0_indices(batch:torch.tensor) -> List[int]:
    '''
    Returns the indices of all "node 0" nodes in the batch
    '''
    batch_size = batch.max().item()+1
    node_index_by_graph = split_node_index_by_graph(batch, batch_size)
    return [x[0][0].item() for x in node_index_by_graph]

# node_index_by_graph = split_node_index_by_graph(b.batch, BATCH_SIZE)
# node0_indices = [x[0][0].item() for x in node_index_by_graph]
# node0_indices
# get_node0_indices(b.batch)

In [169]:
# apparently we can just index into x directly with these indices...
# b.x[node0_indices].shape
# for n0 in b.x[node0_indices]:
#     print(decode_astnode(n0))

# decode_typeseq(b.y)
# decode_typeseq(b.y[None,0,:])
# # dataset.num_classes
# b.y

In [170]:
import torch
from torch.nn import functional as F
from torch_geometric.data import Dataset
from torch_geometric.nn import GATConv, Linear

class TypeSeqModel(torch.nn.Module):
    def __init__(self, dataset:Dataset, hidden_channels:int):
        super(TypeSeqModel, self).__init__()

        # if we go with fewer layers than the # hops in our dataset
        # that may be fine for experimenting, but eventually we are wasting
        # time/space and can cut our dataset down to match (# hops = # layers)
        self.gat1 = GATConv(dataset.num_node_features, hidden_channels)
        self.gat2 = GATConv(hidden_channels, hidden_channels)
        self.gat3 = GATConv(hidden_channels, hidden_channels)

        # TODO - later on, add sequential layer(s) here

        self.pred_head = Linear(hidden_channels, dataset.num_classes)

    def forward(self, x, edge_index, batch):
        node0_indices = get_node0_indices(batch)

        # FIXME: if we need floats in the dataset, go back and save the data
        # this way in our Dataset class
        x = x.to(torch.float32)

        # GNN layers
        # ----------
        # NOTE: it's tempting to downselect to node0 indices here, but I think
        # that may be incorrect - we would not be passing ALL NODES through the
        # network, just node 0.

        # I think we WANT to compute the network on every node
        # for N hops and then simply make predictions based on the node 0 nodes
        #
        # yeah...if we did not compute this on ALL nodes, I think we are
        # making it effectively 1 hop only, and just going multiple rounds with 1 hop!

        h = self.gat1(x, edge_index)
        h = h.relu()

        h = self.gat2(h, edge_index)
        h = h.relu()

        h = self.gat3(h, edge_index)
        # relu here?

        logits = self.pred_head(h[node0_indices])
        return logits


In [171]:
# TODO: training
torch.manual_seed(33)

model = TypeSeqModel(dataset, hidden_channels=64)
print(model)

TypeSeqModel(
  (gat1): GATConv(47, 64, heads=1)
  (gat2): GATConv(64, 64, heads=1)
  (gat3): GATConv(64, 64, heads=1)
  (pred_head): Linear(64, 20, bias=True)
)


In [172]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.CrossEntropyLoss()

def train_one_epoch(train_loader:DataLoader, dev):
    model.train()

    for data in train_loader:
        data.to(dev)
        out = model(data.x, data.edge_index, data.batch)
        # FIXME: change this to floats in Dataset if we need to stick with this
        loss = criterion(out, data.y.to(torch.float32))
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

def eval(loader:DataLoader, dev):
    model.eval()

    correct = 0
    total_loss = 0
    for data in loader:
        data.to(dev)
        out = model(data.x, data.edge_index, data.batch)
        # FIXME: change this to floats in Dataset if we need to stick with this
        loss = criterion(out, data.y.to(torch.float32))
        total_loss += loss.item()
        pred_probabilities = F.softmax(out, dim=1)
        correct += int((pred_probabilities.argmax(dim=1) == data.y.argmax(dim=1)).sum())
    return correct/len(loader.dataset), total_loss/len(loader.dataset)

In [173]:
import torch

if not torch.cuda.is_available():
    print(f'CUDA NOT AVAILABLE!')
else:
    print(f'Device count: {torch.cuda.device_count()}')
    for i in range(torch.cuda.device_count()):
        print(f'Device {i}: {torch.cuda.get_device_name(i)}')

CUDA_DEVICE = 0

Device count: 4
Device 0: Tesla M10
Device 1: Tesla M10
Device 2: Tesla M10
Device 3: Tesla M10


In [174]:
from tqdm.auto import trange

# 2m, 13.7s to get initial train/test accuracy on CPU
# ~3m to get initial " on GPU
# 7m and counting without finishing first epoch using individual .to() methods...

with torch.cuda.device(CUDA_DEVICE):
    cuda_dev = torch.cuda.current_device()
    device = f'cuda:{cuda_dev}' if torch.cuda.is_available() else 'cpu'
    print(f'---------------')
    print(f'Current device: {device}')
    print(f'Current CUDA device: {cuda_dev} ({torch.cuda.get_device_name(cuda_dev)})')

    # move model to device
    model = model.to(device)

    train_acc, train_loss = eval(train_loader, device)
    test_acc, test_loss = eval(test_loader, device)
    print(f'Train loss = {train_loss:.4f}, train accuracy = {train_acc*100:,.2f}%')
    print(f'Test loss = {test_loss:.4f}, test accuracy = {test_acc*100:,.2f}%')

    for epoch in trange(1, 50):
        train_one_epoch(train_loader, device)
        train_acc, train_loss = eval(train_loader, device)
        test_acc, test_loss = eval(test_loader, device)
        # TODO: wandb...
        torch.save(model, 'type_seq_model.pt')

    train_acc, train_loss = eval(train_loader, device)
    test_acc, test_loss = eval(test_loader, device)
    print(f'Train loss = {train_loss:.4f}, train accuracy = {train_acc*100:,.2f}%')
    print(f'Test loss = {test_loss:.4f}, test accuracy = {test_acc*100:,.2f}%')

---------------
Current device: cuda:0
Current CUDA device: 0 (Tesla M10)


Train loss = 0.0475, train accuracy = 3.75%
Test loss = 0.0475, test accuracy = 2.81%


100%|██████████| 49/49 [00:24<00:00,  2.01it/s]


Train loss = 0.0186, train accuracy = 63.59%
Test loss = 0.0177, test accuracy = 65.00%


In [175]:
# for i in range(b.x.shape[0]):
#     print(decode_astnode(b.x[i]))

In [176]:
# rungid, bid, _, _, _ = dataset[0].varid
# [(x.varid[0], x.varid[1]) for x in dataset[10000:10100]][:5]

In [177]:
[(i, x.varid[0], x.varid[1]) for i, x in enumerate(dataset[10000:10100])][:5]

indices = [10000, 10001, 10002, 10003, 10004]
from torch.utils.data import Subset

ss = Subset(dataset, indices)
loader = DataLoader(ss, shuffle=True)
[(x.varid[0][0], x.varid[0][1]) for x in loader]

IndexError: range object index out of range